# Parseurs

Dans ce notebook nous utiliserons le parseur [lxml](http://lxml.de/) qui est un binding de libxml2 et [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) 

## Parser de l'html

Beautiful Soup nous permet de parser simplement du contenu html. Même si le contenu est mal formé, le module bs reconstitue un arbre et offre des fonctions faciles à utiliser pour parcourir l'arbre ou y rechercher des éléments.  
Beautiful Soup n'est pas un parseur, il utilise les parseurs et offre une API simplifiée à ses utilisateurs.

Nous travaillerons directement avec du contenu en ligne à l'aide du module `requests`. Si vous ne l'avez pas en magasin, installez le.  
Décembre c'est le mois des listes, nous nous attacherons à la liste des 100 meilleures chansons de l'année de NPR la radio publique américaine : https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1  
Allez y faire un tour.

In [31]:
import requests
from bs4 import BeautifulSoup

url = "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

Voilà nous avons maintenant un objet `soup` de classe Beautiful Soup.  
La [doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) est très claire.

In [2]:
# je cherche l'élement avec le tag 'title'
print(soup.title)
# le tag de l'élément
print(soup.title.name)
# le contenu textuel de l'élément
print(soup.title.string)

<title>NPR Cookie Consent and Choices</title>
title
NPR Cookie Consent and Choices


Ah mais non. C'est pas ça qu'on veut. Comment faire pour s'épargner cette page de consentement ?

In [34]:
url = "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1"
cookies = {
    'choiceVersion':'1',
    'dateOfChoice':'1607425093937g',
    'trackingChoice':'true'
}
r = requests.get(url, cookies=cookies)
soup = BeautifulSoup(r.text, 'lxml')
# je cherche l'élement avec le tag 'title'
print(soup.title)
# le tag de l'élément
print(soup.title.name)
# le contenu textuel de l'élément
print(soup.title.string)

<title>NPR's 100 Best Songs Of 2020, Ranked : NPR</title>
title
NPR's 100 Best Songs Of 2020, Ranked : NPR


Ah voilà, très bien ces petits cookies.

On cherche à récupérer la liste des 100 chansons : rang, titre, interprète. Puis on les affichera par ordre croissant.  
Il faut inspecter le code source et repérer les élements html et les classes utilisées pour le contentu qui nous intéresse.  
Exemple avec le premier, enfin le 100ème : BTS. Dynamite. 🎶 Cos I… I… I'm in the stars tonight 🎸 🎶

```html
<h6 class="edTag"><a id="bts" class="anchor"> </a>100.</h6>
<h3 class="edTag">BTS</h3>
<h3 class="edTag">"Dynamite"</h3>
```

In [ ]:
# Exemple pour récupérer les éléments h6 class='edTag'
for item in soup.find_all('h6', attrs={'class':'edTag'}):
    print(item.text)
# on peut aussi utiliser la notation suivante
#for item in soup.find_all('h6', class_="edTag"):
#    print(item.text)

### ✍️  Exo ✍️
Maintenant à vous de jouer. Il faut parcourir la [doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) pour trouver la fonction qui vous permettra de récupérer les deux éléments h3 suivants et seulement ceux-là. 
Je vous laisse chercher un peu ![Alt Text](https://media.giphy.com/media/l2SpZkQ0XT1XtKus0/giphy.gif)

Encore un peu ![on cherche](https://media.giphy.com/media/JO9pi3EeHzyBu5YNMK/giphy.gif)

In [ ]:
for item in soup.find_all('h6', attrs={'class':'edTag'}):
    rang = item.text
    h3s = item.find_next_siblings('h3', attrs={'class':'edTag'}, limit=2)
    artist = h3s[0].text
    title = h3s[1].text
    print(f"{rang}, {artist}, {title}")

### ✍️  Exo ✍️

C'est bien mais pas suffisant. Il reste :
  1. nettoyer les rangs, c-a-d supprimmer le point qui traîne à la fin et l'espace des fois.
  2. stocker les infos dans une donnée structurée. Utilisez une classe à vous ou plus simple un `namedtuple`
  3. faire l'opération pour toutes les pages web afin d'avoir le classement de 1 à 100.

In [29]:
urls = [
    "https://www.npr.org/2020/12/03/931771524/the-100-best-songs-of-2020-page-1",
    "https://www.npr.org/2020/12/03/934634561/the-100-best-songs-of-2020-page-2?utm_source=page1&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634607/the-100-best-songs-of-2020-page-3?utm_source=page2&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634855/the-100-best-songs-of-2020-page-4?utm_source=page3&utm_campaign=next&utm_term=bottom&utm_medium=internal",
    "https://www.npr.org/2020/12/03/934634998/the-100-best-songs-of-2020-page-5?utm_source=page4&utm_campaign=next&utm_term=bottom&utm_medium=internal"
]
# à vous

In [46]:
from collections import namedtuple

Song = namedtuple('Song', ['rang', 'artiste', 'titre'])
songs = []
cookies = {
    'choiceVersion':'1',
    'dateOfChoice':'1607425093937g',
    'trackingChoice':'true'
}
for url in urls:
    r = requests.get(url, cookies=cookies)
    soup = BeautifulSoup(r.text, 'lxml')
    for item in soup.find_all('h6', attrs={'class':'edTag'}):
        rang = item.text.rstrip(". ").lstrip()
        h3s = item.find_next_siblings('h3', attrs={'class':'edTag'}, limit=2)
        artiste = h3s[0].text
        titre = h3s[1].text
        songs.append(Song(rang, artiste, titre))

for song in sorted(songs, key=lambda x:int(x.rang)):
    print(song.rang, song.artiste, song.titre)

1 Cardi B (feat. Megan Thee Stallion) "WAP"
2 Christine and the Queens "People, I've been sad"
3 Megan Thee Stallion (feat. Beyoncé) "Savage Remix"
4 Mickey Guyton "Black Like Me"
5 Bad Bunny (feat. Jowell & Randy and Ñengo Flow) "Safaera"
6 Adrianne Lenker "anything"
7 Bob Dylan "Murder Most Foul"
8 Thundercat "Dragonball Durag" 
9 Ana Tijoux "Antifa Dance"
10 Adia Victoria "South Gotta Change"
11 Sam Hunt "Hard to Forget"
12 Jazmine Sullivan "Lost One"
13 Caylee Hammack "Small Town Hypocrite"
14 J Hus (feat. Koffee) "Repeat"
15 Perfume Genius "On The Floor"
16 Chris Stapleton "Starting Over"
17 Phoebe Bridgers "I Know The End"
18 Víkingur Ólafsson "The Arts and the Hours"
19 Joshua Redman, Brad Mehldau, Christian McBride & Brian Blade "Right Back Round Again"
20 SZA (feat. Ty Dolla $ign) "Hit Different" 
21 Lil Baby "The Bigger Picture"
22 Taylor Swift "invisible string"
23 Childish Gambino "47.48"
24 Fiona Apple "I Want You To Love Me"
25 Goodie Mob "4 My Ppl"
26 The Chicks "Gasligh

## Parser de l'xml

Nous allons travailler sur un fichier au format TEI extrait du corpus *Corpus 14*  
PRAXILING - UMR 5267 (PRAXILING) (2014). Corpus 14 [Corpus]. ORTOLANG (Open Resources and TOols for LANGuage) - www.ortolang.fr, https://hdl.handle.net/11403/corpus14/v1.  

Le fichier se nomme ``josephine-1-150119.xml``. Il s'agit d'une lettre d'une femme de soldat à son époux.  
Nous allons extraire du fichier TEI les informations suivantes :  
- titre (``/TEI/teiHeader/fileDesc/titleStmt/title``)
- source (``/TEI/teiHeader/fileDesc/sourceDesc/p``)
- contenu de la lettre (``/TEI/text/body``)

### Avec lxml

Pourquoi `lxml` et pas `xml.etree.ElementTree` ? Parce que : [1](http://lxml.de/intro.html) et surtout [2](http://lxml.de/performance.html)  
La bonne nouvelle c'est que votre code sera aussi compatible avec `xml.etree.ElementTree` ou `xml.etree.cElementTree` parce que xml utilise l'API ElementTree. Sauf pour la méthode `xpath` qui est propre à `libxml`.

In [57]:
from lxml import etree
tree = etree.parse('data/josephine-1-150119.xml')
root = tree.getroot()

# Parcours des enfants de la racine (commentaires et éléments)
for child in root:
    print(child.tag)

<cyfunction Comment at 0x7ff63423aef0>
{http://www.tei-c.org/ns/1.0}teiHeader
<cyfunction Comment at 0x7ff63423aef0>
{http://www.tei-c.org/ns/1.0}facsimile
{http://www.tei-c.org/ns/1.0}text


Le fichier utilise l'espace de nom TEI : ``<TEI xmlns="http://www.tei-c.org/ns/1.0">``, nous devrons l'indiquer dans nos instructions de recherche.  
Voyons ça pour le titre (``/TEI/teiHeader/fileDesc/titleStmt/title``)

In [58]:
# la méthode find renvoie le premier élément qui correspond au chemin argument (ElementPath et non Xpath)
title = root.find("./tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
print("Tag : {}".format(title.tag))
print("Texte : {}".format(title.text))

Tag : {http://www.tei-c.org/ns/1.0}title
Texte : Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues


Même traitement pour la source :

In [59]:
source = root.find("./tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
print("Tag : {}".format(source.tag))
print("Texte : {}".format(source.text))

Tag : {http://www.tei-c.org/ns/1.0}p
Texte : Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


lxml a aussi une méthode ``xpath`` qui permet d'utiliser directement des expressions xpath (sans oublier les espace de noms pour notre fichier) :

In [11]:
source = root.xpath("/tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p", namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
print(type(source)) #xpath retourne une liste
print(source[0].text)
#ou bien
source = root.xpath("/tei:TEI/tei:teiHeader/tei:fileDesc/tei:sourceDesc/tei:p/text()", namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
print(source[0])

<class 'list'>
Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.
Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


Pour le contenu il faut ruser. La difficulté ici tient à l'utilisation d'élements `<lb/>` de type 'milestones' pour noter les retours à la ligne :  
```xml
<p>
je reponse a ton aimableux lettres<lb/>
que nous a fait plaisir en naprenas<lb/>
que tu et enbonne santes car il<lb/>
anais de maime pour nous<lb/>
</p>
```

In [116]:
# la méthode findall renvoie une liste avec tous les éléments correspondant au chemin argument
body = root.findall("./tei:text/tei:body/tei:p", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
for elem in body:
    print(elem.text)


cher Laurent

je reponse a ton aimableux lettres

cher Laurent je repons a la cartes


Ici on ne récupère que les noeuds text précédant les éléments `<lb/>`  
Une requête `xpath` va nous permettre de récupérer tous les noeuds text

In [12]:
body = root.xpath("//tei:text/tei:body//text()", namespaces={'tei':"http://www.tei-c.org/ns/1.0"})
for text in body:
    print(text, end="")



Baillargues Le 19 janvier 1915


cher Laurent


je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous


cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire partire
partir un paquet quil aillae
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len verras
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Josep

## avec DOM

L'API `ElementTree` est propre à Python, `DOM` est une API indépendante d'un langage de programmation. Il existe des implémentations `DOM` dans la plupart des langages de programmation modernes.  

In [60]:
from xml.dom import minidom
dom = minidom.parse("data/josephine-1-150119.xml")
# l'objet Document
dom

In [61]:
title = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'title')[0] # un seul élément 'title' dans le document
print(title) # title est un objet Element, pour accèder au contenu textuel il faut récupérer le noeud texte
print(title.lastChild.nodeName)
print(title.lastChild.nodeValue)

<DOM Element: title at 0x7ff627a61910>
#text
Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues


idem pour la source, sauf qu'on ne peut pas se permettre de rechercher tous les éléments `p`.  
Il faut trouver l'élément `p` fils de `sourceDesc`

In [62]:
sourceDesc = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'sourceDesc')[0]
for node in sourceDesc.childNodes:
    if node.localName == "p":
        print(node.lastChild.nodeValue)

Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault.


Et maintenant le contenu et ses éléments milestones.  
Pour garder la forme vous réécrirez les boucles `for` suivies de `if` en listes en compréhension.

In [63]:
body = dom.getElementsByTagNameNS("http://www.tei-c.org/ns/1.0", 'body')[0]
for node in body.childNodes:
    if node.localName == "p" or "opener":
        for in_node in node.childNodes:
            if in_node.nodeName == "#text":
                print(in_node.nodeValue, end="")


Baillargues Le 19 janvier 1915

cher Laurent

je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous

cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire 
partir un paquet quil 
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len 
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Josephine Pouchet
bien le bo

## Avec lxml et Beautiful Soup

In [49]:
from bs4 import BeautifulSoup

with open("data/josephine-1-150119.xml") as fp:
    soup = BeautifulSoup(fp, 'lxml')

In [50]:
soup.title.text

'Joséphine Pouchet à son époux le 19-01-1915 depuis Baillargues'

In [51]:
soup.sourcedesc.p.text

"Correspondance de Joséphine Pouchet, numérisée par les Archives Départementales de l'Hérault."

Pour le contenu de la lettre il y a la merveilleuse fonction `get_text()`

In [52]:
soup.get_text?

In [92]:
text = soup.find('text')
print(text.getText())





Baillargues Le 19 janvier 1915


cher Laurent


je reponse a ton aimableux lettres
que nous a fait plaisir en naprenas
que tu et enbonne santes car il
anais de maime pour nous


cher Laurent je repons a la cartes
de ma mère quelles et venue au
jourdhui pour de faire partire
partir un paquet quil aillae
les chosette sausice chocolas une
paire de chosette pour Louis
je pense que vous magerè ensenbleus
tu feras repons a la maison te
suite que tu rese vras le paquet
je te dirais que ten le midi il
fait frois il fait du vent glasais
et toi au pas de calais tu nous
dit quil pleus mai tu nous parles
pas si tu a ases pour te garendir
du froit cil te maque quelles chose

tu nas que ledire quon de len verras
verras tu nous dit que charles ta
Ecrie et ta soeux et ta dit que
je lui et envoiez la photot
plurien a te dire pour le moment
que de ten voiez une grose
carriese de tous et boutounase
de ton petit enge 
adorè Albert encorre une foi
te plui Milles baisées te tous
ta fenme pour la vie
Jos

lxml est rapide, Beautiful Soup simple à utiliser. Le combo diablement efficace.

Il y a un autre module super pour le web que nous ne verrons pas ici mais que je me dois de vous indiquer : https://selenium-python.readthedocs.io/  
Selenium va vous permettre d'automatiser des actions sur un navigateur. Je vous conseille d'essayer, c'est assez plaisant de voir votre navigateur piloté par un script.